# 딥드림
* 합성곱 신경망이 학습한 표현을 사용하여 예술적으로 이미지를 조작하는 기법
* 특정 필터가 아니라 전체 층의 활성화를 최대화한다.
* 한꺼번에 많은 특성을 넣어 시각화한다.
* 입력 이미지를 여러 다른 스케일로 처리한다.

In [26]:
from PIL import Image

# 이미지 파일 경로 설정
image_path = "./3rd_project/test.jpg" 

# 이미지 열기
image = Image.open(image_path)

# 이미지 크기 확인
width, height = image.size
print(f"이미지 크기: {width} x {height}")

이미지 크기: 1920 x 1080


In [27]:
# 이미지 크기 299*299로 맞추기
from PIL import Image

# 이미지 파일 경로 설정
image_path = "./3rd_project/test.jpg"

# 이미지 열기
image = Image.open(image_path)

# 이미지 크기를 299x299로 축소
resized_image = image.resize((299, 299))
# 축소된 이미지 저장
resized_image.save("./3rd_project/299_test.jpg")
print("이미지 축소 완료: 299x299 크기로 저장되었습니다.")


이미지 축소 완료: 299x299 크기로 저장되었습니다.


In [28]:
from tensorflow import keras
import numpy as np

def get_img_array(img_path, target_size):
    # Load and preprocess the image
    try:
        img = keras.utils.load_img(img_path, target_size=target_size)
        array = keras.utils.img_to_array(img)
        array = np.expand_dims(array, axis=0)  # Adds a batch dimension
        array = keras.applications.xception.preprocess_input(array)  # Preprocess for Xception
        return array
    except Exception as e:
        print(f"Error while processing the image: {e}")
        return None

# Example usage
base_image_path = "./3rd_project/299_test.jpg"  # Adjust your local path
target_size = (224, 224)  # Target image size
img_array = get_img_array(base_image_path, target_size)

In [29]:
# 사전 훈련된 컨브넷 사용
# InceptionV3 모델 로드
from tensorflow.keras.applications import inception_v3

model = inception_v3.InceptionV3(weights ="imagenet", include_top =False)

In [30]:
# 중간층의 활성화를 반환하는 특성 추출 모델
# 경사 상승법 단계 동안에 최대화할 손실에 대한 각 층의 기여도에 가중치를 주기 위해 스칼라 값을 선택

layer_settings = {
    "mixed4": 1.0,
    "mixed5": 1.5,
    "mixed6": 2.0,
    "mixed7": 2.5,
}
outputs_dict = dict(
    [
        (layer.name, layer.output)
        for layer in [model.get_layer(name) for name in layer_settings.keys()]
    ]
)
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

In [31]:
# 딥드림 손실 - 각 스케일마다 최대화
# 모든 필터 활성화를 동시에 최대화
def compute_loss(input_image):
    features = feature_extractor(input_image)
    loss = tf.zeros(shape=())
    for name in features.keys():
        coeff = layer_settings[name]
        activation = features[name]
        loss += coeff * tf.reduce_mean(tf.square(activation[:, 2:-2, 2:-2, :]))
    return loss


In [32]:
# 각 옥타브에서 실행할 경사 상승법법
import tensorflow as tf

@tf.function
def gradient_ascent_step(image, learning_rate):
    with tf.GradientTape() as tape:
        tape.watch(image)
        loss = compute_loss(image)
    grads = tape.gradient(loss, image)
    grads = tf.math.l2_normalize(grads)
    image += learning_rate * grads
    return loss, image


def gradient_ascent_loop(image, iterations, learning_rate, max_loss=None):
    for i in range(iterations):
        loss, image = gradient_ascent_step(image, learning_rate)
        if max_loss is not None and loss > max_loss:
            break
        print(f"... 스텝 {i}에서 손실 값: {loss:.2f}")
    return image

In [50]:
step = 50.
num_octave = 2
octave_scale = 1.4
iterations = 10
max_loss = 15.

In [51]:
# 이미지 처리
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(image_path)
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.inception_v3.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img.shape[1], img.shape[2], 3))
    img += 1.0
    img *= 127.5
    img = np.clip(img, 0, 255).astype("uint8")
    return img

In [52]:
# MKL(Math Kernel Library)을 활성화하도록 설정
# MKL의 역할 : MKL은 Intel의 고성능 수학 라이브러리로, TensorFlow 연산 속도를 최적화하는 데 사용
# 특히 CPU 기반 시스템에서 행렬 연산, 컨볼루션 등과 같은 수학적 계산을 더 빠르게 처리합니다.
import os
os.environ["TF_DISABLE_MKL"] = "0"


In [53]:
original_img = preprocess_image(base_image_path)
original_shape = original_img.shape[1:3]

successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)
successive_shapes = successive_shapes[::-1]

shrunk_original_img = tf.image.resize(original_img, successive_shapes[0])

img = tf.identity(original_img)
for i, shape in enumerate(successive_shapes):
    print(f"{shape} 크기의 {i}번째 옥타브 처리")
    img = tf.image.resize(img, shape)
    img = gradient_ascent_loop(
        img, iterations=iterations, learning_rate=step, max_loss=max_loss
    )
    upscaled_shrunk_original_img = tf.image.resize(shrunk_original_img, shape)
    same_size_original = tf.image.resize(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = tf.image.resize(original_img, shape)

keras.utils.save_img("./3rd_project/t1.png", deprocess_image(img.numpy()))

(213, 213) 크기의 0번째 옥타브 처리


c:\Users\acorn\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_933']
Received: inputs=Tensor(shape=(1, 213, 213, 3))
  warnings.warn(msg)


... 스텝 0에서 손실 값: 1.08
... 스텝 1에서 손실 값: 1.00
... 스텝 2에서 손실 값: 2.56
... 스텝 3에서 손실 값: 8.17
(299, 299) 크기의 1번째 옥타브 처리


c:\Users\acorn\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_933']
Received: inputs=Tensor(shape=(1, 299, 299, 3))
  warnings.warn(msg)
